In [4]:
!pip install keras

In [5]:
import warnings
warnings.filterwarnings("ignore")

**Load Libraries**

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import *
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import keras
import numpy as np
import matplotlib.pyplot as plt

In [7]:
(X_train, _),(_,_) = mnist.load_data()

In [8]:
X_train.shape

(60000, 28, 28)

In [9]:
X_train = (X_train-127.5)/127.5

print(X_train.min())
print(X_train.max())

-1.0
1.0


In [10]:
TOTAL_EPOCHS=10
BATCH_SIZE=256
HALF_BATCH=128
NO_OF_BATCHES=int(X_train.shape[0]/BATCH_SIZE)
NOISE_DIM=100
adam = Adam(learning_rate=2e-4, beta_1=0.5)

**Generator Model: Upsampling**

In [11]:
generator=Sequential()
generator.add(Dense(units=7*7*128, input_shape=(NOISE_DIM,)))
generator.add(Reshape((7,7,128)))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

#(7,7,128) -> (14,14,64)

generator.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

#(14,14,64) -> (28,28,1)

generator.add(Conv2DTranspose(1,(3,3),strides=(2,2),padding='same', activation='tanh'))
generator.compile(loss=keras.losses.binary_crossentropy, optimizer=adam)
generator.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 6272)                │         633,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 7, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 14, 14, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 28, 28, 1)           │             577 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 708,609 (2.70 MB)

 Trainable params: 708,225 (2.70 MB)

 Non-trainable params: 384 (1.50 KB)

In [12]:
# Discriminator Model -> i.e Down Sampling
#(28,28,1) -> (14,14,64)

discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same',input_shape=(28,28,1)))
discriminator.add(LeakyReLU(0.2))

#(14,14,64) -> (7,7,128)

discriminator.add(Conv2D(128, kernel_size=(3,3), strides=(2,2), padding='same'))
discriminator.add(LeakyReLU(0.2))

#(7,7,128) -> 6272
discriminator.add(Flatten())
discriminator.add(Dense(100))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(1,activation='sigmoid'))

discriminator.compile(loss=keras.losses.binary_crossentropy,optimizer=adam)
discriminator.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 14, 14, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 7, 7, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 100)                 │         627,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 701,897 (2.68 MB)

 Trainable params: 701,897 (2.68 MB)

 Non-trainable params: 0 (0.00 B)

**Combined Model & Functional API**

In [13]:
#Combined Model
discriminator.trianable=False
gan_input=Input(shape= (NOISE_DIM, ))
generated_img = generator(gan_input)
gan_output=discriminator(generated_img)

#Functional API
model=Model(gan_input, gan_output)
model.compile(loss=keras.losses.binary_crossentropy, optimizer=adam)

In [14]:
model.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 28, 28, 1)           │         708,609 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 1)                   │         701,897 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,410,506 (5.38 MB)

 Trainable params: 1,410,122 (5.38 MB)

 Non-trainable params: 384 (1.50 KB)

In [15]:
X_train=X_train.reshape(-1,28,28,1)
X_train.shape

(60000, 28, 28, 1)

In [16]:
def display_images(samples=25):
    # Correct the shape of the noise input
    noise = np.random.normal(0, 1, size=(samples, NOISE_DIM))
    generated_img = generator.predict(noise)

    # Plot the generated images
    plt.figure(figsize=(10, 10))
    for i in range(samples):
        plt.subplot(5, 5, i + 1)
        plt.imshow(generated_img[i].reshape(28, 28), cmap="binary")
        plt.axis('off')

    plt.tight_layout()  # Add this for better spacing
    plt.show()

In [ ]:
# Ensure the discriminator is compiled with a separate optimizer
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
)

# Compile the combined model (generator + discriminator)
discriminator.trainable = False  # Freeze discriminator for generator training
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
)

# Training loop
d_losses = []
g_losses = []
for epoch in range(TOTAL_EPOCHS):
    epoch_d_loss = 0.0
    epoch_g_loss = 0.0

    # Mini batch gradient descent
    for step in range(NO_OF_BATCHES):
        ### Step 1: Train Discriminator
        discriminator.trainable = True

        # Get real data
        idx = np.random.randint(0, 60000, HALF_BATCH)
        real_imgs = X_train[idx]

        # Get fake data
        noise = np.random.normal(0, 1, size=(HALF_BATCH, NOISE_DIM))
        fake_imgs = generator.predict(noise)

        # Labels
        real_y = np.ones((HALF_BATCH, 1)) * 0.9  # Label smoothing
        fake_y = np.zeros((HALF_BATCH, 1))

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_imgs, real_y)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_y)
        d_loss = 0.5 * (d_loss_real + d_loss_fake)
        epoch_d_loss += d_loss

        ### Step 2: Train Generator
        discriminator.trainable = False

        noise = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_DIM))
        ground_truth_y = np.ones((BATCH_SIZE, 1))

        # Train generator through the combined model
        g_loss = model.train_on_batch(noise, ground_truth_y)
        if isinstance(g_loss, list):  # Check if the result is a list
             g_loss = g_loss[0]  # Use the primary loss
        epoch_g_loss += g_loss


    # Log the losses
    print(f"Epoch {epoch + 1}, Disc loss: {epoch_d_loss / NO_OF_BATCHES}, Generator loss: {epoch_g_loss / NO_OF_BATCHES}")
    d_losses.append(epoch_d_loss / NO_OF_BATCHES)
    g_losses.append(epoch_g_loss / NO_OF_BATCHES)

    # Save the generator model periodically
    if (epoch + 1) % 10 == 0:
        generator.save("generator.h5")
        display_images()

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━